In [ ]:
%pip install peft
%pip install trl
%pip install dataset -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os
import gc
import torch
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model
from trl import CPOConfig, CPOTrainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Using device: cuda
GPU: Tesla T4
GPU Memory: 14.7 GB


In [ ]:
dataset = load_dataset("thainq107/Vi-Alpaca-Preference")
print("Mẫu đầu tiên:", dataset['train'][0])


train_cpo_ds = dataset['train'].select(range(45000))
test_cpo_ds = dataset['test'].select(range(500))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/506 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Mẫu đầu tiên: {'id': 'alpaca-7294', 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.', 'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.', 'rejected': 'Câu này không có lỗi ngữ pháp.'}


In [ ]:

def format_for_cpo(dataset):
    formatted_samples = []
    for d in dataset:
        formatted_samples.append({
            'prompt': d['question'].rstrip(),
            'chosen': '\n' + d['chosen'].lstrip(),
            'rejected': '\n' + d['rejected'].lstrip()
        })
    return Dataset.from_list(formatted_samples)
train_cpo_ds = format_for_cpo(train_cpo_ds)
test_cpo_ds = format_for_cpo(test_cpo_ds)

In [ ]:
BASE_MODEL_NAME = "hoa12356/Llama-3.2-1B-Instruct-Chat-sft-hoa"
REWARD_MODEL_PATH = "hoa12356/vietnamese-reward-model"
OUTPUT_MODEL_PATH = "./rlhf_cpo_ckpts"

reward_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_PATH, padding_side="left")
if reward_tokenizer.pad_token is None:
    reward_tokenizer.add_special_tokens({"pad_token": "[PAD]"})


reward_tokenizer.model_max_length = 512
reward_tokenizer.chat_template = getattr(reward_tokenizer, "chat_template", None)

print("Loading reward model...")
reward_model = AutoModelForSequenceClassification.from_pretrained(
    REWARD_MODEL_PATH,
    num_labels=1,
    torch_dtype=torch.float16,
    device_map="auto"
)

reward_model.config.pad_token_id = reward_tokenizer.pad_token_id
reward_model.eval()


tokenizer_config.json:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Loading reward model...


adapter_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at hoa12356/Llama-3.2-1B-Instruct-Chat-sft-hoa and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=512, bias=False)
            (lora_dropout): M

In [ ]:
def compute_scores(batch):
    """
    Compute reward scores for chosen and rejected responses using GPU
    """
    batch_size = len(batch["chosen"])

    chosen_inputs = reward_tokenizer(
        batch["chosen"],
        padding=True,
        truncation=True,
        max_length=reward_tokenizer.model_max_length,
        return_tensors='pt'
    )

    rejected_inputs = reward_tokenizer(
        batch["rejected"],
        padding=True,
        truncation=True,
        max_length=reward_tokenizer.model_max_length,
        return_tensors='pt'
    )

    chosen_inputs = {k: v.to(device) for k, v in chosen_inputs.items()}
    rejected_inputs = {k: v.to(device) for k, v in rejected_inputs.items()}

    with torch.no_grad():
        try:
            score_chosen = reward_model(**chosen_inputs).logits.squeeze(-1)
            score_rejected = reward_model(**rejected_inputs).logits.squeeze(-1)
        except torch.cuda.OutOfMemoryError:
            print("GPU OOM encountered, trying smaller batch...")

            score_chosen_list = []
            score_rejected_list = []

            chunk_size = max(1, batch_size // 2)
            for i in range(0, batch_size, chunk_size):
                end_idx = min(i + chunk_size, batch_size)

                chosen_chunk = {k: v[i:end_idx] for k, v in chosen_inputs.items()}
                rejected_chunk = {k: v[i:end_idx] for k, v in rejected_inputs.items()}

                score_chosen_chunk = reward_model(**chosen_chunk).logits.squeeze(-1)
                score_rejected_chunk = reward_model(**rejected_chunk).logits.squeeze(-1)

                score_chosen_list.append(score_chosen_chunk)
                score_rejected_list.append(score_rejected_chunk)

            score_chosen = torch.cat(score_chosen_list, dim=0)
            score_rejected = torch.cat(score_rejected_list, dim=0)
    return {
        "score_chosen": score_chosen.cpu().tolist(),
        "score_rejected": score_rejected.cpu().tolist()
    }

In [ ]:
def filter_problematic_examples(dataset, tokenizer):
    """
    Remove examples where truncation causes chosen vs. rejected
    token lengths to differ by >1 token.
    """
    def is_valid(example):

        enc_chosen = tokenizer(
            example["chosen"],
            truncation=True,
            max_length=tokenizer.model_max_length
        )
        enc_reject = tokenizer(
            example["rejected"],
            truncation=True,
            max_length=tokenizer.model_max_length
        )
        return abs(len(enc_chosen.input_ids) - len(enc_reject.input_ids)) <= 1

    return dataset.filter(is_valid)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()


print("Computing scores for training dataset...")

train_cpo_ds_bulk = train_cpo_ds.map(
    compute_scores,
    batched=True,
    batch_size=4,
    desc="Computing training scores"
)

print("Computing scores for test dataset...")
test_cpo_ds_bulk = test_cpo_ds.map(
    compute_scores,
    batched=True,
    batch_size=4,
    desc="Computing test scores"
)

# Clear GPU cache after processing
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Dataset processing completed!")
print(f"Training samples: {len(train_cpo_ds_bulk)}")
print(f"Test samples: {len(test_cpo_ds_bulk)}")


Computing scores for training dataset...


Computing training scores:   0%|          | 0/45000 [00:00<?, ? examples/s]

Computing scores for test dataset...


Computing test scores:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset processing completed!
Training samples: 45000
Test samples: 500


In [ ]:
train_cpo_ds_bulk.save_to_disk("train_cpo_ds_bulk_processed")
test_cpo_ds_bulk.save_to_disk("test_cpo_ds_bulk_processed")


Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
policy_tok = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, padding_side="left")
if policy_tok.pad_token is None:
    policy_tok.add_special_tokens({"pad_token": "[PAD]"})
policy_tok.max_length = 128
policy_tok.chat_template = getattr(policy_tok, "chat_template", None)

tokenizer_config.json:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
train_cpo_ds = filter_problematic_examples(train_cpo_ds_bulk, policy_tok)
test_cpo_ds  = filter_problematic_examples(test_cpo_ds_bulk,  policy_tok)

Filter:   0%|          | 0/45000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
train_cpo_ds.take(3).to_pandas()

,prompt,chosen,rejected,score_chosen,score_rejected
0,Tạo một tiêu đề hấp dẫn về một công nghệ mới.,"\n""Điện toán hàng không: Công nghệ đột phá tiê...","\n""Robot thông minh nhân tạo: Hướng tới tương ...",-2.880859,-3.376953
1,"Tìm từ giống nhất về nghĩa với ""thịnh vượng"".\...",\nphát đạt,\nđạp.,-0.280273,-0.299805
2,Viết lại câu sau với cùng chủ ngữ và động từ n...,\nTôi sẽ ăn sáng.,\nTôi ăn bánh.,-1.783203,-1.740234


In [ ]:
class CPOTrainerWithLoRA:
    """
    Trainer that wraps TRL's CPOTrainer with LoRA (PEFT) configuration.
    """
    def __init__(
        self,
        model_name: str,
        train_dataset: Dataset,
        eval_dataset: Dataset,
        output_dir: str,
        lora_r: int = 6,
        lora_alpha: int = 32,
        lora_dropout: float = 0.05,
        target_modules=None,
        per_device_train_batch_size: int = 1,
        per_device_eval_batch_size: int = 4,
        num_train_epochs: int = 3,
        seed: int = 42,
        report_to: str = "none"
    ):
        self.model_name     = model_name
        self.train_dataset  = train_dataset
        self.eval_dataset   = eval_dataset
        self.output_dir     = output_dir
        self.seed           = seed

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({"pad_token": "[PAD]"})
        self.tokenizer.max_length = 128
        self.tokenizer.chat_template = getattr(self.tokenizer, "chat_template", None)

        self.policy = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
        self.policy.config.pad_token_id = self.tokenizer.pad_token_id

        self.peft_config = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            target_modules=target_modules or ["q_proj"],
            bias="none",
            task_type="CAUSAL_LM"
        )
        self.policy = get_peft_model(self.policy, self.peft_config)

        self.cpo_config = CPOConfig(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            num_train_epochs=num_train_epochs,
            remove_unused_columns=False,
            seed=seed,
            report_to=report_to
        )

        self.trainer = CPOTrainer(
            model=self.policy,
            args=self.cpo_config,
            processing_class=self.tokenizer,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            peft_config=self.peft_config
        )

    def train(self):
        """Run the CPO training loop."""
        self.trainer.train()

    def save(self, save_directory: str = None):
        """Save the fine-tuned policy model and tokenizer."""
        target_dir = save_directory or self.output_dir
        os.makedirs(target_dir, exist_ok=True)
        self.policy.save_pretrained(target_dir)
        self.tokenizer.save_pretrained(target_dir)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = CPOTrainerWithLoRA(
    model_name=BASE_MODEL_NAME,
    train_dataset=train_cpo_ds,
    eval_dataset=test_cpo_ds,
    output_dir=OUTPUT_MODEL_PATH,
)

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/1951 [00:00<?, ? examples/s]

Map:   0%|          | 0/1951 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/1951 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss
500,2.957900
1000,2.945200
1500,2.780200
2000,2.782000
2500,2.828600
3000,2.929200
3500,2.688300
4000,2.845900
4500,2.830900
5000,2.797900


In [ ]:
trainer.save()

In [ ]:
from huggingface_hub import login, upload_folder

login()

In [ ]:
trainer.policy.push_to_hub("hoa12356/CPO_trainning")
trainer.tokenizer.push_to_hub("hoa12356/CPO_trainning")

adapter_model.safetensors:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hoa12356/CPO_trainning/commit/f55e516ddffd512ce61b6fc6de7c5cfbd5d324a0', commit_message='Upload tokenizer', commit_description='', oid='f55e516ddffd512ce61b6fc6de7c5cfbd5d324a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hoa12356/CPO_trainning', endpoint='https://huggingface.co', repo_type='model', repo_id='hoa12356/CPO_trainning'), pr_revision=None, pr_num=None)